# Import & Mount

In [ ]:
pip install split-folders

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
import splitfolders
import os   
import cv2 
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import pickle
from PIL import Image
import pandas as pd
from tqdm import tqdm
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 준비 

In [ ]:
col=["Title", "Style","Gender", "Season"]

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/이미지인식 프로젝트/이미지마이닝_통합.csv", names = col)

In [ ]:
# 성별 - 남자 

title_man = df['Title'][df['Gender'] == 0].tolist()

for name in tqdm( title_man ):
    try:
        im = Image.open("/content/drive/MyDrive/이미지인식 프로젝트/최종/{}.jpg".format(name))        
        im.save('images/man/{}.jpg'.format(name))
    except:
        pass



100%|██████████| 3221/3221 [07:28<00:00,  7.18it/s]


In [ ]:
# 성별 - 여자 

title_woman = df['Title'][df['Gender'] == 1].tolist()


for name in tqdm( title_woman ):
    try:
        im = Image.open("/content/drive/MyDrive/이미지인식 프로젝트/최종/{}.jpg".format(name))        
        im.save('images/woman/{}.jpg'.format(name))
    except:
        pass



100%|██████████| 1892/1892 [04:13<00:00,  7.47it/s]


# Preprocessing

In [ ]:
folder_path = '/content/images'   
label_names = os.listdir(folder_path)  
label_names.remove('.ipynb_checkpoints')
label_names

['man', 'woman']

In [ ]:
# 각 이미지 경로 만들어주기 

dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):  
    sub_path = folder_path+'/'+label+'/'  
    dataset[label] = []  
    for filename in os.listdir(sub_path): 
        dataset[label].append(sub_path+filename)
dataset

{'man': ['/content/images/man/formal_7_당찬 아우라.jpg',
  '/content/images/man/dandy_3_베이식한 조합.jpg',
  '/content/images/man/man_0_9891.jpg',
  '/content/images/man/formal_10_스타일 만점.jpg',
  '/content/images/man/dandy_6_훈훈한 이미지.jpg',
  '/content/images/man/man_0_7197.jpg',
  '/content/images/man/dandy_10_세련된 레이어링.jpg',
  '/content/images/man/casual_1_휴양지 스타일.jpg',
  '/content/images/man/dandy_4_봄기운 물씬.jpg',
  '/content/images/man/sports_6_남자의 스포츠.jpg',
  '/content/images/man/man_0_1903.jpg',
  '/content/images/man/sports_6_모두 입어줘!.jpg',
  '/content/images/man/casual_16_파스텔 컬러 연출법.jpg',
  '/content/images/man/man_0_4096.jpg',
  '/content/images/man/casual_4_시원한 게 좋아.jpg',
  '/content/images/man/man_0_1831.jpg',
  '/content/images/man/dandy_14_차분한 룩.jpg',
  '/content/images/man/man_0_6380.jpg',
  '/content/images/man/dandy_14_센스 있어.jpg',
  '/content/images/man/sports_11_애슬레저!.jpg',
  '/content/images/man/casual_33_남다른 캐주얼 룩.jpg',
  '/content/images/man/man_0_7435.jpg',
  '/content/images/man/m

In [ ]:
!mkdir resized

In [ ]:
# 이미지 resize 후 resized폴더에 저장 

!mkdir resized/man
!mkdir resized/woman

In [ ]:
dataset.items()

dict_items([('man', ['/content/images/man/formal_7_당찬 아우라.jpg', '/content/images/man/dandy_3_베이식한 조합.jpg', '/content/images/man/man_0_9891.jpg', '/content/images/man/formal_10_스타일 만점.jpg', '/content/images/man/dandy_6_훈훈한 이미지.jpg', '/content/images/man/man_0_7197.jpg', '/content/images/man/dandy_10_세련된 레이어링.jpg', '/content/images/man/casual_1_휴양지 스타일.jpg', '/content/images/man/dandy_4_봄기운 물씬.jpg', '/content/images/man/sports_6_남자의 스포츠.jpg', '/content/images/man/man_0_1903.jpg', '/content/images/man/sports_6_모두 입어줘!.jpg', '/content/images/man/casual_16_파스텔 컬러 연출법.jpg', '/content/images/man/man_0_4096.jpg', '/content/images/man/casual_4_시원한 게 좋아.jpg', '/content/images/man/man_0_1831.jpg', '/content/images/man/dandy_14_차분한 룩.jpg', '/content/images/man/man_0_6380.jpg', '/content/images/man/dandy_14_센스 있어.jpg', '/content/images/man/sports_11_애슬레저!.jpg', '/content/images/man/casual_33_남다른 캐주얼 룩.jpg', '/content/images/man/man_0_7435.jpg', '/content/images/man/man_0_130.jpg', '/content/images/

In [ ]:
for label, filenames in dataset.items():  
    for filename in filenames:
        try:
            img = cv2.imread(filename)

            percent = 1
            if(img.shape[1] > img.shape[0]) :      
                percent = 224/img.shape[1]
            else :
                percent = 224/img.shape[0]

            img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)

            y,x,h,w = (0,0,img.shape[0], img.shape[1])


            w_x = (224-(w-x))/2  
            h_y = (224-(h-y))/2

            if(w_x < 0):        
                w_x = 0
            elif(h_y < 0):
                h_y = 0

            M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
            img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  # 패딩 

            # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
            cv2.imwrite('/content/resized/{0}/{1}'.format(label, filename.split("/")[-1]) , img_re)
        except:
            pass

# train, validation, test set 분할

In [ ]:
splitfolders.ratio('resized', output='dataset', seed=77, ratio=(0.6, 0.2, 0.2))  

Copying files: 7827 files [00:01, 6753.82 files/s]


In [ ]:
folder_path = '/content/dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'man': ['/content/dataset/train/man/formal_7_당찬 아우라.jpg',
  '/content/dataset/train/man/man_0_9891.jpg',
  '/content/dataset/train/man/formal_10_스타일 만점.jpg',
  '/content/dataset/train/man/dandy_6_훈훈한 이미지.jpg',
  '/content/dataset/train/man/dandy_10_세련된 레이어링.jpg',
  '/content/dataset/train/man/casual_1_휴양지 스타일.jpg',
  '/content/dataset/train/man/man_0_1903.jpg',
  '/content/dataset/train/man/sports_6_모두 입어줘!.jpg',
  '/content/dataset/train/man/casual_16_파스텔 컬러 연출법.jpg',
  '/content/dataset/train/man/casual_4_시원한 게 좋아.jpg',
  '/content/dataset/train/man/dandy_14_차분한 룩.jpg',
  '/content/dataset/train/man/sports_11_애슬레저!.jpg',
  '/content/dataset/train/man/man_0_7435.jpg',
  '/content/dataset/train/man/man_0_130.jpg',
  '/content/dataset/train/man/dandy_4_산뜻한 멋.jpg',
  '/content/dataset/train/man/dandy_18_시크한 매력.jpg',
  '/content/dataset/train/man/dandy_5_데이트 룩.jpg',
  '/content/dataset/train/man/man_0_2828.jpg',
  '/content/dataset/train/man/casual_3_여행 가는 기분.jpg',
  '/content/dataset/tr

In [ ]:
label_gender = {'man':0, 'woman':1}

In [ ]:
x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) 

        x_train.append(image)
        y_train.append(label_gender[label])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = x_train.astype('float32')

In [ ]:
x_train.shape, y_train.shape

((4695, 224, 224, 3), (4695,))

# train set 증강 및 형변환

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,    # 사다리꼴로 
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
folder_path

'/content/dataset/train'

In [ ]:
for label in  os.listdir(folder_path):
    label_path = folder_path + '/' + label + '/'
    for filename in os.listdir(label_path): 
        filepath = label_path + filename

        img = load_img(filepath)

        x = img_to_array(img)

        x = x.reshape((1,) + x.shape)  # batch size = 1

        i = 0

        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=label_path, save_prefix=label, save_format='jpg'):
            i += 1
            if i > 2:   
                break  

In [ ]:
folder_path = '/content/dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'man': ['/content/dataset/train/man/formal_7_당찬 아우라.jpg',
  '/content/dataset/train/man/man_0_1216.jpg',
  '/content/dataset/train/man/man_0_9891.jpg',
  '/content/dataset/train/man/formal_10_스타일 만점.jpg',
  '/content/dataset/train/man/man_0_6045.jpg',
  '/content/dataset/train/man/dandy_6_훈훈한 이미지.jpg',
  '/content/dataset/train/man/man_0_2522.jpg',
  '/content/dataset/train/man/man_0_7197.jpg',
  '/content/dataset/train/man/dandy_10_세련된 레이어링.jpg',
  '/content/dataset/train/man/man_0_1094.jpg',
  '/content/dataset/train/man/man_0_6226.jpg',
  '/content/dataset/train/man/man_0_7513.jpg',
  '/content/dataset/train/man/man_0_8896.jpg',
  '/content/dataset/train/man/man_0_6356.jpg',
  '/content/dataset/train/man/casual_1_휴양지 스타일.jpg',
  '/content/dataset/train/man/man_0_2508.jpg',
  '/content/dataset/train/man/man_0_4945.jpg',
  '/content/dataset/train/man/man_0_3816.jpg',
  '/content/dataset/train/man/man_0_327.jpg',
  '/content/dataset/train/man/man_0_1082.jpg',
  '/content/dataset/train

In [ ]:
label_gender = {'man':0, 'woman':1}

In [ ]:
x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_train.append(image)
        y_train.append(label_gender[label]) 

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = x_train.astype('float32')

In [ ]:
x_train.shape, y_train.shape # 이미지 수 늘어남 

((13450, 224, 224, 3), (13450,))

# validation, test set 형변환

In [ ]:
folder_path = '/content/dataset/val'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'man': ['/content/dataset/val/man/dandy_4_봄기운 물씬.jpg',
  '/content/dataset/val/man/man_0_4096.jpg',
  '/content/dataset/val/man/man_0_6380.jpg',
  '/content/dataset/val/man/casual_33_남다른 캐주얼 룩.jpg',
  '/content/dataset/val/man/man_0_6345.jpg',
  '/content/dataset/val/man/sports_9_편하게 입자!.jpg',
  '/content/dataset/val/man/sports_8_궁극의 애슬레저.jpg',
  '/content/dataset/val/man/casual_34_잘 어울려.jpg',
  '/content/dataset/val/man/man_0_3161.jpg',
  '/content/dataset/val/man/dandy_6_모노 톤 스타일링.jpg',
  '/content/dataset/val/man/man_0_8929.jpg',
  '/content/dataset/val/man/sports_6_때로는 이렇게.jpg',
  '/content/dataset/val/man/man_0_1196.jpg',
  '/content/dataset/val/man/casual_21_감각적이야.jpg',
  '/content/dataset/val/man/man_0_4964.jpg',
  '/content/dataset/val/man/man_0_8549.jpg',
  '/content/dataset/val/man/man_0_7052.jpg',
  '/content/dataset/val/man/man_0_8675.jpg',
  '/content/dataset/val/man/man_0_5821.jpg',
  '/content/dataset/val/man/sports_2_캠핑족은 주목!.jpg',
  '/content/dataset/val/man/man_0_132

In [ ]:
x_val, y_val = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_val.append(image)
        y_val.append(label_gender[label]) # label을 index로 변경

In [ ]:
x_val, y_val= np.array(x_val), np.array(y_val)

In [ ]:
x_val = x_val.astype('float32')

In [ ]:
x_val.shape, y_val.shape

((1564, 224, 224, 3), (1564,))

In [ ]:
folder_path = '/content/dataset/test'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'man': ['/content/dataset/test/man/dandy_3_베이식한 조합.jpg',
  '/content/dataset/test/man/man_0_7197.jpg',
  '/content/dataset/test/man/sports_6_남자의 스포츠.jpg',
  '/content/dataset/test/man/man_0_1831.jpg',
  '/content/dataset/test/man/dandy_14_센스 있어.jpg',
  '/content/dataset/test/man/man_0_8220.jpg',
  '/content/dataset/test/man/sports_6_운동하는 훈남.jpg',
  '/content/dataset/test/man/man_0_7485.jpg',
  '/content/dataset/test/man/sports_7_캐주얼 셋업.jpg',
  '/content/dataset/test/man/man_0_4093.jpg',
  '/content/dataset/test/man/man_0_3817.jpg',
  '/content/dataset/test/man/man_0_3098.jpg',
  '/content/dataset/test/man/man_0_5344.jpg',
  '/content/dataset/test/man/casual_7_시티보이 감성.jpg',
  '/content/dataset/test/man/man_0_3864.jpg',
  '/content/dataset/test/man/sports_5_힘이 느껴져!.jpg',
  '/content/dataset/test/man/sports_2_원 마일 웨어 연출.jpg',
  '/content/dataset/test/man/sports_1_바캉스 패션.jpg',
  '/content/dataset/test/man/casual_19_데일리 패션.jpg',
  '/content/dataset/test/man/man_0_5545.jpg',
  '/content/dat

In [ ]:
x_test, y_test = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_test.append(image)
        y_test.append(label_gender[label]) # label을 index로 변경

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)

In [ ]:
x_test = x_test.astype('float32')

In [ ]:
x_test.shape, y_test.shape

((1568, 224, 224, 3), (1568,))

# Zero-centering

In [ ]:
def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0) 

In [ ]:
zero_mean_img = zero_mean(x_train)

In [ ]:
zero_mean_img.shape

(224, 224, 3)

In [ ]:
x_train -= zero_mean_img

In [ ]:
x_val -= zero_mean_img
x_test -= zero_mean_img

# Transfer Learning

In [ ]:
## 일부 재학습 모델
base_model = ResNet50(include_top=False, input_shape = (224, 224,3), weights = 'imagenet')

for layer in base_model.layers[:-10]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='softmax')(x)  

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


# train loss, validation loss 시각화 

In [ ]:
def tr_plot(tr_data, start_epoch):
    #Plot the training and validation data
    tacc=tr_data.history['accuracy']
    tloss=tr_data.history['loss']
    vacc=tr_data.history['val_accuracy']
    vloss=tr_data.history['val_loss']
    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(vloss)#  this is the epoch with the lowest validation loss
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    acc_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,8))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout
    #plt.style.use('fivethirtyeight')
    plt.show()

# 모델 학습 및 평가 

In [ ]:
tr_plot(model.fit(x_train, y_train, epochs = 5, batch_size= 125, validation_data=(x_val, y_val)), 0)

In [ ]:
model.evaluate(x_test, y_test)